# Freed RG Lab — receipts and plots

This notebook collects receipts from `analysis/freed/` and `.tau_ledger/freed/`,
renders simple plots (matplotlib), and prints an axioms-style summary.


In [ ]:
import os, glob, json, math\ntry:\n    import matplotlib.pyplot as plt\nexcept Exception as e:\n    plt = None; print('[info] matplotlib not available:', e)\n\ndef latest(pattern):\n    files = sorted(glob.glob(pattern))\n    return files[-1] if files else None\n\ndef load_json(path):\n    try:\n        with open(path, 'r', encoding='utf-8') as f: return json.load(f)\n    except Exception as e:\n        print('[warn] cannot load', path, e); return None\n\nos.makedirs('analysis/freed', exist_ok=True)\n

## Flatness: trace/logdet identity error

In [ ]:
det_path = latest('analysis/freed/*_determinant_identity.json')\nif det_path:\n    det = load_json(det_path)\n    xs = [row['ell'] for row in det.get('grid',[])]\n    ys = [abs(row['lhs_tr_identity'] - row['rhs_fd']) for row in det.get('grid',[])]\n    if plt and xs and ys:\n        plt.figure(figsize=(6,3))\n        plt.plot(xs, ys)\n        plt.yscale('log')\n        plt.xlabel('ell')\n        plt.ylabel('abs error (log)')\n        plt.title('flatness receipt')\n        plt.tight_layout(); plt.show()\n    print('max_abs_err =', det.get('max_abs_err'))\nelse:\n    print('[info] no determinant_identity receipt found')\n

## Factorization (phi off/on) and TMF deltas

In [ ]:
phi_off = latest('analysis/freed/*_factorization_phi_off.json')\nphi_on  = latest('analysis/freed/*_factorization_phi_on.json')\nfor label, path in [('phi_off', phi_off), ('phi_on', phi_on)]:\n    if not path: print('[info] no factorization receipt for', label); continue\n    data = load_json(path)\n    print(label, 'sum_segments=', data.get('sum_segments'), 'whole=', data.get('whole'), 'abs_err=', data.get('abs_err'))\n\ntmf_path = latest('analysis/freed/*_tmf_deltas.json')\nif tmf_path:\n    t = load_json(tmf_path); tmf = t.get('tmf', {})\n    for key in ('phi_off','phi_on'):\n        node = tmf.get(key);\n        if not node: continue\n        print(key, 'delta_whole=', node.get('delta_whole'), 'delta_segments=', node.get('delta_segments'))\nelse:\n    print('[info] no TMF deltas receipt found')\n

## Eta-term and defects

In [ ]:
eta_path = latest('analysis/freed/eta_tmf_*.json')\nif eta_path:\n    e = load_json(eta_path)\n    stk = e.get('stack', {})\n    print('eta_half_logW =', stk.get('eta_half_logW'), 'effective_stack =', stk.get('effective'))\nelse:\n    print('[info] no eta report found')\n\ndef_path = latest('analysis/freed/defect_*.json')\nif def_path:\n    d = load_json(def_path)\n    print('defects sign_flip_phase=', d.get('sign_flip_phase'), 'perm_phase=', d.get('perm_phase'))\nelse:\n    print('[info] no defect report found')\n

## Axioms summary (latest values by certificate)

In [ ]:
import glob\nledgers = sorted(glob.glob('.tau_ledger/freed/*.manifest.json'))\ndef pick(cert_key):\n    for p in reversed(ledgers):\n        try:\n            with open(p,'r',encoding='utf-8') as f: m=json.load(f)\n            c=m.get('certificates',{})\n            if cert_key in c: return m.get('run_id'), c[cert_key]\n        except Exception: pass\n    return None, None\nrows=[\n ('flatness',)+pick('determinant_trace_identity_max_abs_err'),\n ('stack_invariance',)+pick('monodromy_shape_norm_err'),\n ('factorization_phi_off',)+pick('factorization_phi_off_abs_err'),\n ('factorization_phi_on',)+pick('factorization_phi_on_abs_err'),\n ('eta_B5',)+pick('eta_half_logW_B5'),\n ('eta_E6',)+pick('eta_half_logW_E6'),\n ('defect_flip_err',)+pick('sign_flip_shape_err'),\n ('defect_perm_err',)+pick('perm_shape_err'),\n]\nprint('Axioms (name, run_id, value):')\nfor r in rows: print('  -', r)\n